## Info

In [ ]:
# Contact: Patrick.Jantz@nau.edu
# Material based on tutorials that can be found in resources linked below

## Resources

In [ ]:
# https://geemap.org/
# https://developers.google.com/earth-engine/guides/python_install-conda#linux_5
# https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api
# https://geemap.org/notebooks/12_zonal_statistics/
# https://geemap.org/notebooks/13_zonal_statistics_by_group/
# https://geemap.org/notebooks/36_quality_mosaic/
# https://geemap.org/notebooks/34_extract_values/

## Imports

In [ ]:
# Imports
import os
import ee
import pandas as pd
import altair as alt
import numpy as np
import geemap
import bqplot
from bqplot import pyplot as plt

## Initialize

In [ ]:
#ee.Authenticate()
ee.Initialize()

## Basic Plotting

In [ ]:
aa = np.array([[1,2,3],[4,5,6]])
print(aa)
bb = np.array([[1,1,1],[1,1,1]])
print(aa+bb)
x = np.arange(100)
source = pd.DataFrame({
  'x': x,
  'f(x)': np.sin(x / 5)
})
print(source)
alt.Chart(source).mark_line().encode(
    x='x',
    y='f(x)'
)

## Mapping

In [ ]:
# Subset target ecoregion
# Target ecoregion ID
# colorado plateau 429
# arizona mountains 346
ecoid = 429
#aoi = (ee.FeatureCollection("RESOLVE/ECOREGIONS/2017").filter(ee.Filter.eq('ECO_ID', ecoid))).geometry()
aoi = (ee.FeatureCollection("RESOLVE/ECOREGIONS/2017").filter(ee.Filter.eq('ECO_ID', ecoid)))

In [ ]:
Map = geemap.Map(center=(40, -100), zoom=4)
Map

In [ ]:
# Add dem to map
dem = ee.Image('USGS/SRTMGL1_003')
dem_vis = {
'min': 0,
'max': 4000,
'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}
Map.addLayer(dem, dem_vis, 'SRTM DEM', True, 0.5)

In [ ]:
# Add ecoregion to map
Map.addLayer(aoi, {}, 'Ecoregion')

In [ ]:
# Terraclimate
met = ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE')
bandNames = met.first().bandNames()
print('Band names: ', bandNames.getInfo())
# Get properties
properties = met.first().propertyNames()
print('Metadata properties: ', properties.getInfo())

In [ ]:
# geemap method for image properties
image_props = geemap.image_props(met.first())
print(image_props.getInfo())

In [ ]:
# Terraclimate metrics of interest
metrics = ['pr','pet','aet','vpd','vap','tmmx']
metricsN = ['PR','PET','AET','VPD','VAP','TMMX']
# scaling factors [1,0.1,0.1,0.01,0.001,0.1]

# Filter by metrics and date
start_date = '1958-01-01'
end_date = '2021-12-31'
met = ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE').filterDate(start_date, end_date).select(metrics)

In [ ]:
bandNames = met.first().bandNames()
print('Band names: ', bandNames.getInfo())

In [ ]:
# Visualize first image
tmmx_vis = {
'min': 0,
'max': 300,
'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}
Map.addLayer(met.first().select('tmmx'), tmmx_vis, 'TMAX', True, 0.5)


## Processing and Graphing

In [ ]:
# Output directory for zonal stats
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
met_stats = os.path.join(out_dir, 'met_stats.csv')
print(out_dir)
print(met_stats)

In [ ]:
# Allowed output formats: csv, shp, json, kml, kmz
# Allowed statistics type: MEAN, MAXIMUM, MINIMUM, MEDIAN, STD, MIN_MAX, VARIANCE, SUM
geemap.zonal_statistics(met, aoi, met_stats, statistics_type='MEAN', scale=10000, crs='EPSG:4269')

In [ ]:
mStats = pd.read_csv(met_stats)
print(mStats.shape)
print(list(mStats))

In [ ]:
# Get zstat rows
zsI = [i for i,j in enumerate(list(mStats)) if any(x in j for x in ['pr','pet','aet','vpd','vap'])]
mStats = mStats.iloc[:,zsI]
mStats = mStats.T


In [ ]:
print(mStats.index)
print(list(mStats))

mStats = mStats.rename(columns={0: 'Value'})
mStats['Year'] = [i.split('_')[0][0:4] for i in mStats.index]
mStats['Month'] = [int(i.split('_')[0][4:6]) for i in mStats.index]
mStats['YearMonth'] = [i.split('_')[0] for i in mStats.index]
mStats['Variable'] = [i.split('_')[1] for i in mStats.index]
mStats = mStats.reset_index(drop=True)
mStats.head()

In [ ]:
# Get vpd
vpd = mStats.loc[mStats['Variable'] == 'vpd'].copy() # Compare without copy.
# Scale
vpd.Value = vpd.Value * 0.01
vpd = vpd.reset_index(drop=True)
# Create timestamp
vpd['Timestamp'] = pd.to_datetime(vpd.YearMonth, format='%Y%m', errors='ignore')
print(vpd.head())

In [ ]:
# Line plot
fig = plt.figure(title="VPD")
line_chart = plt.plot(x=vpd.Timestamp, y=vpd.Value)

plt.xlabel("Date")
plt.ylabel("VPD")

plt.show()

In [ ]:
# Bar chart
cs = 'blueorange'
bchart = alt.Chart(vpd).mark_bar(size=1).encode(
    x='Timestamp:T',
    y='Value:Q',
    color=alt.Color(
        'Value:Q', scale=alt.Scale(scheme=cs, domain=(0, 3))),
    tooltip=[
        alt.Tooltip('Timestamp:T', title='Date'),
        alt.Tooltip('Value:Q', title='VPD')
    ]).properties(width=800, height=200) # can try .interactive()
bchart

In [ ]:
dchart = alt.Chart(vpd).mark_rect().encode(
    x='Year:O',
    y='Month:O',
    color=alt.Color(
        'mean(Value):Q', scale=alt.Scale(scheme=cs, domain=(0, 3))),
    tooltip=[
        alt.Tooltip('Year:O', title='Year'),
        alt.Tooltip('Month:O', title='Month'),
        alt.Tooltip('mean(Value):Q', title='VPD')
    ]).properties(width=800, height=200)
dchart

In [ ]:
# Get ppt
ppt = mStats.loc[mStats['Variable'] == 'pr'].copy()
ppt = ppt.reset_index(drop=True)
# Create timestamp
ppt['Timestamp'] = pd.to_datetime(ppt.YearMonth, format='%Y%m', errors='ignore')
print(ppt.head())

In [ ]:
# Simple bar chart aggregated by year
alt.Chart(ppt).mark_bar().encode(
    x='Year',
    y='sum(Value)'
).properties(width=800, height=200)

In [ ]:
# Bar chart
cs = 'redblue'
bchart = alt.Chart(ppt).mark_bar(size=6).encode(
    x=alt.X('Year:T',axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('sum(Value):Q', title='PPT'),
    color=alt.Color(
        'sum(Value):Q', scale=alt.Scale(scheme=cs, domain=(0, 400)),legend=alt.Legend(title="PPT(mm)")),
    tooltip=[
        alt.Tooltip('Year:T', title='Year'),
        alt.Tooltip('sum(Value):Q', title='PPT')
    ]).properties(width=800, height=200)
bchart

## Export/Import Data

In [ ]:
# Start interactive map
Map = geemap.Map()
Map

In [ ]:
# Subset target ecoregion
# Target ecoregion ID
# colorado plateau 429
# arizona mountains 346
ecoid = 429
#aoi = (ee.FeatureCollection("RESOLVE/ECOREGIONS/2017").filter(ee.Filter.eq('ECO_ID', ecoid))).geometry()
aoi = (ee.FeatureCollection("RESOLVE/ECOREGIONS/2017").filter(ee.Filter.eq('ECO_ID', ecoid)))

In [ ]:
# Greenest pixel mosaic
start_date = '2019-01-01'
end_date = '2019-12-31'

l8 = (
    ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA')
    .filterBounds(aoi)
    .filterDate(start_date, end_date)
)

median = l8.median()

visParams = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 0.4,
}

Map.addLayer(median, visParams, 'Median')

In [ ]:
# Functions
def addNDVI(image):
    ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    return image.addBands(ndvi)
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())
def addMonth(image):
    img_date = ee.Date(image.date())
    img_doy = ee.Number.parse(img_date.format('M'))
    return image.addBands(ee.Image(img_doy).rename('month').toInt())
def addDOY(image):
    img_date = ee.Date(image.date())
    img_doy = ee.Number.parse(img_date.format('D'))
    return image.addBands(ee.Image(img_doy).rename('doy').toInt())


In [ ]:
# Add ndvi and date info
withNDVI = l8.map(addNDVI).map(addDate).map(addMonth).map(addDOY)
# Quality mosaic based on ndvi
greenest = withNDVI.qualityMosaic('NDVI')
# Get band info
greenest.bandNames().getInfo()
ndvi = greenest.select('NDVI')
palette = [
    '#d73027',
    '#f46d43',
    '#fdae61',
    '#fee08b',
    '#d9ef8b',
    '#a6d96a',
    '#66bd63',
    '#1a9850',
]
Map.addLayer(ndvi, {'palette': palette}, 'NDVI')

In [ ]:
# Add month layer
Map.addLayer(
    greenest.select('month'),
    {'palette': ['red', 'blue'], 'min': 1, 'max': 12},
    'Greenest month',
)

In [ ]:
# Set output dir and filename
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
filename = os.path.join(out_dir, 'ndvi_mosaic.tif')
# Clip and export
image = ndvi.clip(aoi).unmask()
geemap.ee_export_image(
    image, filename=filename, scale=10000, region=aoi.geometry(), file_per_band=False
)

In [ ]:
# Not working yet but works via gui
# This seems to load raster to the map as a non-gee layer
#Map.add_raster(filename, bands=1, colormap='terrain', layer_name='Imported NDVI')